In [1]:
import json

tweets_data = []
tweets_file = open('geotagged_tweets_20160812-0912.jsons', "r")
print(tweets_file)

counter = 0
for line in tweets_file:
    counter = counter + 1
    if counter > 30000:
        break
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue
print(tweets_data[1:10])

<_io.TextIOWrapper name='geotagged_tweets_20160812-0912.jsons' mode='r' encoding='cp1252'>
[{'created_at': 'Fri Aug 12 10:04:02 +0000 2016', 'id': 764039733076897792, 'id_str': '764039733076897792', 'text': '@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN COLLUSION TOGETHER \n\n#NOJUSTICE \n\n@realDonaldTrump \n#TrumpPence \n\nhttps://t.co/5GMNZq40V3', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': 813286, 'in_reply_to_user_id_str': '813286', 'in_reply_to_screen_name': 'BarackObama', 'user': {'id': 82496193, 'id_str': '82496193', 'name': 'Red Octopus', 'screen_name': 'redoctapus', 'location': 'Ogletown, DE', 'url': None, 'description': 'AWSCWI Pipefitter USAF NRA I remember USA, Hunting Fishing, HarleyDavidsons- a time when we HS students could carry guns in our trks 2 school', 'protected': False, 'verified': False, 'followers_count

In [2]:
import os
import json
import numpy as np
import pandas as pd
import nltk
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import rgb2hex
from descartes import PolygonPatch
from shapely.geometry import Polygon, MultiPolygon
import seaborn as sns
import sklearn
import gensim, logging, warnings, spacy
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel

In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
stop_words = stopwords.words('english')
stop_words.extend(['from', 'amp', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

In [5]:
from pandas.io.json import json_normalize
# Thank gawd for this package - all json fields straight into columns in a dataframe, no bullshit
tweets_df = json_normalize(tweets_data)

tweets_df.head()

,contributors,coordinates,coordinates.coordinates,coordinates.type,created_at,entities.hashtags,entities.media,entities.symbols,entities.urls,entities.user_mentions,...,user.profile_sidebar_fill_color,user.profile_text_color,user.profile_use_background_image,user.protected,user.screen_name,user.statuses_count,user.time_zone,user.url,user.utc_offset,user.verified
0,None,NaN,NaN,NaN,Fri Aug 12 10:04:00 +0000 2016,[],"[{'id': 764039718237409281, 'id_str': '7640397...",[],[],"[{'screen_name': 'theblaze', 'name': 'TheBlaze...",...,EFEFEF,333333,True,False,DUPUY77,17620,None,http://ggm-dupuy.com,NaN,False
1,None,NaN,NaN,NaN,Fri Aug 12 10:04:02 +0000 2016,"[{'text': 'NOJUSTICE', 'indices': [62, 72]}, {...",NaN,[],"[{'url': 'https://t.co/5GMNZq40V3', 'expanded_...","[{'screen_name': 'BarackObama', 'name': 'Barac...",...,000000,000000,False,False,redoctapus,5046,Eastern Time (US & Canada),None,-14400.0,False
2,None,NaN,NaN,NaN,Fri Aug 12 10:04:10 +0000 2016,[],"[{'id': 764039762973896704, 'id_str': '7640397...",[],[],"[{'screen_name': 'theblaze', 'name': 'TheBlaze...",...,EFEFEF,333333,True,False,DUPUY77,17621,None,http://ggm-dupuy.com,NaN,False
3,None,NaN,NaN,NaN,Fri Aug 12 10:04:21 +0000 2016,[],NaN,[],[],"[{'screen_name': 'HillaryClinton', 'name': 'Hi...",...,99CC33,3E4415,True,False,jjorgi30,2231,Melbourne,None,36000.0,False
4,None,NaN,NaN,NaN,Fri Aug 12 10:04:30 +0000 2016,"[{'text': 'CNN', 'indices': [0, 4]}, {'text': ...",NaN,[],[],[],...,F3F3F3,333333,True,False,bspence5,92605,None,None,NaN,False


In [6]:
cols = []

for col in tweets_df.columns:
    if col in ('created_at', 'place.country','place.country_code','place.name','is_quote_status','lang','text','user.created_at','user.description','user.name','user.location','place.place_type', 'user-screen_name', 'in_reply_to_screen_name','geo.type','geo.coordinates', 'retweeted_status-text'):
        cols.append(col)

neat_df = tweets_df.loc[:, cols]
neat_df.head()

,created_at,geo.coordinates,geo.type,in_reply_to_screen_name,is_quote_status,lang,place.country,place.country_code,place.name,place.place_type,text,user.created_at,user.description,user.location,user.name
0,Fri Aug 12 10:04:00 +0000 2016,NaN,NaN,theblaze,False,und,United States,US,Frontenac,city,@theblaze @realDonaldTrump https://t.co/TY9DlZ...,Fri Sep 02 14:54:17 +0000 2011,"Fashion photographer, love action and adventur...",Miami,GIL DUPUY
1,Fri Aug 12 10:04:02 +0000 2016,NaN,NaN,BarackObama,False,en,United States,US,Baton Rouge,city,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,Thu Oct 15 00:28:04 +0000 2009,"AWSCWI Pipefitter USAF NRA I remember USA, Hun...","Ogletown, DE",Red Octopus
2,Fri Aug 12 10:04:10 +0000 2016,NaN,NaN,theblaze,False,und,United States,US,Frontenac,city,@theblaze @realDonaldTrump https://t.co/n050DB...,Fri Sep 02 14:54:17 +0000 2011,"Fashion photographer, love action and adventur...",Miami,GIL DUPUY
3,Fri Aug 12 10:04:21 +0000 2016,NaN,NaN,HillaryClinton,False,en,Australia,AU,Melbourne,city,@HillaryClinton he will do in one year all the...,Tue Jun 02 01:59:59 +0000 2009,The best movies are written and directed by pe...,Australia,JanJorgenson
4,Fri Aug 12 10:04:30 +0000 2016,NaN,NaN,None,False,en,United States,US,Baltimore,city,#CNN #newday clear #Trump deliberately throwin...,Mon Aug 20 09:43:48 +0000 2012,"Retired, political wonk mother of one grandmot...",None,Beverly Spence


In [7]:
import re
#probably gotta clean it more but this is a first step
def clean_text(dataframe, col, bigrams = False):
    # make new column for accounts
    dataframe.loc[:, "accounts"] = dataframe[col].str.findall(r'@.*?(?=\s|$)')
    # make new column for hashtag
    dataframe.loc[:, "hashtag"] = dataframe[col].str.findall(r'#.*?(?=\s|$)')
    #remove links
    dataframe.loc[:, col] = dataframe[col].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
    #remove @
    dataframe.loc[:, col] = dataframe[col].str.replace(r'@.*?(?=\s|$)'," ")
    #remove #
    dataframe.loc[:, col] = dataframe[col].str.replace(r'#.*?(?=\s|$)'," ")
    #remove special characters
    dataframe.loc[:, col] = dataframe[col].str.replace("\W"," ")
    #remove digits
    dataframe.loc[:, col] = dataframe[col].str.replace('\d+', '')
    

    return dataframe
neat_df = clean_text(neat_df, "text")
neat_df.tail(50)

,created_at,geo.coordinates,geo.type,in_reply_to_screen_name,is_quote_status,lang,place.country,place.country_code,place.name,place.place_type,text,user.created_at,user.description,user.location,user.name,accounts,hashtag
29950,Sat Aug 13 17:57:34 +0000 2016,NaN,NaN,realDonaldTrump,True,en,United States,US,Beekman,city,knows he is losing big,Sun Mar 06 02:18:28 +0000 2016,None,None,Elizabeth,[@realDonaldTrump],[]
29951,Sat Aug 13 17:57:39 +0000 2016,NaN,NaN,VLRAmyCurtis,False,en,United States,US,Orange,city,You fill the seats as best you can,Wed Aug 03 04:10:57 +0000 2016,None,"Orange, CA",Dave Spurlock,"[@VLRAmyCurtis, @HeyTammyBruce, @HillaryClinton]",[]
29952,Sat Aug 13 17:57:41 +0000 2016,NaN,NaN,None,False,en,United States,US,Pennsylvania,admin,DeadBroke Fr WH Made MIL Only Was SlushF...,Wed May 16 18:32:59 +0000 2012,Our Time Is Short To Restore America's Truth H...,USA,Born American,[],"[#HillaryClinton, #Charity, #ClintonFoundation..."
29953,Sat Aug 13 17:57:45 +0000 2016,NaN,NaN,None,False,en,United States,US,Wisconsin,admin,Are Dem s the ONLY Charitable people It s b...,Wed Apr 06 04:29:52 +0000 2011,"Wisconsin Liberal 4 Life Proud to be a Wife,Mo...",Way up north Wisconsin,☮JacquiUpNorth☮,[],[#ImWithHer]
29954,Sat Aug 13 17:57:48 +0000 2016,NaN,NaN,realDonaldTrump,True,en,United States,US,Virginia,admin,His hair has been nearly as white as of ...,Wed Sep 09 19:38:53 +0000 2009,"I'm just passing by, adding my thoughts to som...",Frozen Food Aisle,MaryToGo,"[@realDonaldTrump, @hughhewitt, @CNN]",[#FireTrumpIn4Words]
29955,Sat Aug 13 17:57:51 +0000 2016,NaN,NaN,None,False,en,United States,US,Fountain Hill,city,Read amp RT She is a con artist too Perfec...,Wed Mar 28 00:58:05 +0000 2012,"Social Worker, Proud PA Dutch, passionate libe...","Pennsylvania, USA",Amy Snyder,[],[#NeverTrump]
29956,Sat Aug 13 17:57:52 +0000 2016,NaN,NaN,CidaZeeb,False,en,United States,US,New Jersey,admin,yea she defended the rapist of a yr old th...,Tue Oct 23 01:50:02 +0000 2012,"I am a Christian,","New Jersey, USA",redhead4645,"[@CidaZeeb, @HillaryClinton]",[]
29957,Sat Aug 13 17:57:55 +0000 2016,NaN,NaN,ABC,False,en,United States,US,Brandywine,city,But won t,Sun Oct 23 00:37:43 +0000 2011,Writer and veteran journalist.,USA,Satinderpal Kapur,"[@ABC, @realDonaldTrump]",[]
29958,Sat Aug 13 17:58:01 +0000 2016,NaN,NaN,None,False,en,United States,US,Pennsylvania,admin,DeadBroke Fr WH Made MIL Only Was SlushF...,Wed May 16 18:32:59 +0000 2012,Our Time Is Short To Restore America's Truth H...,USA,Born American,[],"[#HillaryClinton, #Charity, #ClintonFoundation..."
29959,Sat Aug 13 17:58:03 +0000 2016,NaN,NaN,realDonaldTrump,False,en,United States,US,Manhattan,city,from a REAL presidential family,Wed Sep 05 11:27:50 +0000 2012,"Hotel Sales Leader, @Q&A Hotel by @FurnishedQ ...",NYC,AnnaGaffney,"[@realDonaldTrump, @IvankaTrump]",[#DonaldTrumpJr]


In [8]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# Create a set of stopwords
stop = set(stopwords.words('english'))

# Create a set of punctuation words 
exclude = set(string.punctuation) 

# This is the function makeing the lemmatization
lemma = WordNetLemmatizer()

# In this function we perform the entire cleaning
def clean(tweet):
    stop_free = " ".join([i for i in tweet.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

# These are the clean tweets.
neat_df['clean_tweet'] = neat_df.text.apply(clean)
neat_df['clean_tweet'].head()

0                                                     
1                                   collusion together
2                                                     
3                            one year thing done eight
4    clear deliberately throwing race knew destabil...
Name: clean_tweet, dtype: object

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 5000, # keep top 1000 terms 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(neat_df['clean_tweet'])

X.shape # check shape of the document-term matrix

(30000, 5000)

In [10]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)

len(svd_model.components_)

20

In [11]:
terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
        print(" ")

Topic 0: 
trump
 
amp
 
tax
 
hillary
 
vote
 
people
 
return
 
Topic 1: 
amp
 
volunteer
 
area
 
help
 
elected
 
nov
 
retweet
 
Topic 2: 
today
 
pressure
 
rain
 
wind
 
hpa
 
orchard
 
forecast
 
Topic 3: 
tax
 
return
 
release
 
pay
 
hiding
 
email
 
released
 
Topic 4: 
trump
 
tax
 
volunteer
 
area
 
nov
 
elected
 
retweet
 
Topic 5: 
hillary
 
clinton
 
crooked
 
obama
 
foundation
 
isi
 
email
 
Topic 6: 
vote
 
america
 
great
 
people
 
make
 
need
 
maga
 
Topic 7: 
like
 
look
 
great
 
america
 
make
 
say
 
people
 
Topic 8: 
great
 
america
 
make
 
say
 
need
 
think
 
love
 
Topic 9: 
say
 
people
 
think
 
sarcasm
 
lie
 
medium
 
believe
 
Topic 10: 
think
 
sarcasm
 
vote
 
right
 
isi
 
obama
 
like
 
Topic 11: 
say
 
vote
 
great
 
isi
 
like
 
know
 
said
 
Topic 12: 
love
 
amp
 
dead
 
dx
 
breakng
 
medrecrds
 
want
 
Topic 13: 
need
 
love
 
want
 
help
 
president
 
say
 
elected
 
Topic 14: 
need
 
amp
 
medium
 
sarcasm
 
clinton
 
dead
 
lie
 
To